In [2]:
import pandas as pd
import numpy as np
import os, sys
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from tqdm.auto import tqdm
import json
sys.path.append('../')
from models.knobs import rocksdb_knobs_make_dict

In [2]:
PATH_internal = "internal"
PATH_external = "external"
PATH_knobs = "rocksdb_conf"
wk_len = 16

In [3]:
knobs = rocksdb_knobs_make_dict(PATH_knobs)
knobs = pd.DataFrame(data=knobs['data'].astype(np.float32), columns=knobs['columnlabels'])
columns = knobs.columns
knobs.head(4)

,max_background_compactions,max_background_flushes,write_buffer_size,max_write_buffer_number,min_write_buffer_number_to_merge,compaction_pri,compaction_style,level0_file_num_compaction_trigger,level0_slowdown_writes_trigger,level0_stop_writes_trigger,...,open_files,block_size,cache_index_and_filter_blocks,max_bytes_for_level_base,max_bytes_for_level_multiplier,target_file_size_base,target_file_size_multiplier,num_levels,memtable_bloom_size_ratio,compression_ratio
0,6.0,16.0,1304576.0,3.0,2.0,1.0,0.0,3.0,24.0,44.0,...,10000.0,9216.0,0.0,5242880.0,11.0,1800192.0,1.0,8.0,0.10,0.67
1,2.0,14.0,911360.0,6.0,2.0,0.0,0.0,7.0,18.0,55.0,...,100000.0,15360.0,1.0,3145728.0,10.0,1729536.0,1.0,5.0,0.15,0.50
2,8.0,6.0,1474560.0,6.0,1.0,2.0,0.0,5.0,18.0,48.0,...,1000000.0,6144.0,0.0,4194304.0,9.0,1429504.0,1.0,6.0,0.05,0.36
3,12.0,8.0,550912.0,7.0,1.0,3.0,0.0,4.0,18.0,35.0,...,10000.0,3072.0,1.0,2097152.0,11.0,900096.0,1.0,7.0,0.05,0.63


In [4]:
print("KNOB NAME \t\t\t\t sum of value counts")
for col in columns:
    print(f"{col:40}\t{len(knobs[[col]].value_counts())}")

KNOB NAME 				 sum of value counts
max_background_compactions              	16
max_background_flushes                  	16
write_buffer_size                       	1536
max_write_buffer_number                 	7
min_write_buffer_number_to_merge        	2
compaction_pri                          	4
compaction_style                        	1
level0_file_num_compaction_trigger      	7
level0_slowdown_writes_trigger          	17
level0_stop_writes_trigger              	33
compression_type                        	4
bloom_locality                          	2
open_files                              	4
block_size                              	15
cache_index_and_filter_blocks           	2
max_bytes_for_level_base                	7
max_bytes_for_level_multiplier          	5
target_file_size_base                   	1537
target_file_size_multiplier             	2
num_levels                              	4
memtable_bloom_size_ratio               	5
compression_ratio                       	101


In [55]:
index_value = dict()
for col in columns:
    iv = knobs[[col]].value_counts()#.reset_index(level=0).drop(columns=0)
    iv = iv.sort_index()
    index_value[col] = pd.Series(data=range(len(iv)), index=iv.index)

In [56]:
index_value

{'max_background_compactions': max_background_compactions
 1.0                            0
 2.0                            1
 3.0                            2
 4.0                            3
 5.0                            4
 6.0                            5
 7.0                            6
 8.0                            7
 9.0                            8
 10.0                           9
 11.0                          10
 12.0                          11
 13.0                          12
 14.0                          13
 15.0                          14
 16.0                          15
 dtype: int64,
 'max_background_flushes': max_background_flushes
 1.0                        0
 2.0                        1
 3.0                        2
 4.0                        3
 5.0                        4
 6.0                        5
 7.0                        6
 8.0                        7
 9.0                        8
 10.0                       9
 11.0                      10
 12

In [ ]:
knobs_one_hot = torch.Tensor()
for i in tqdm(range(len(knobs))):   
    sample = torch.Tensor()
    for col in columns:
        knob_one_hot = torch.zeros(len(index_value[col]))
        knob_one_hot[index_value[col][knobs[col][i]]] = 1
        sample = torch.cat((sample, knob_one_hot))
    sample = sample.unsqueeze(0)
    knobs_one_hot = torch.cat((knobs_one_hot, sample))
knobs_one_hot.shape
np.save('knobsOneHot.npy', np.array(knobs_one_hot))

In [ ]:
np.load('knobsOneHot.npy')

In [5]:
PATH_internal = "internal"
PATH_external = "external"
wk_len = 16
internal_dict = {}

pruned_im = pd.read_csv(os.path.join(PATH_internal, 'internal_ensemble_pruned_tmp.csv'), index_col=0)
for wk in range(wk_len):
    im = pd.read_csv(os.path.join(PATH_internal, f'internal_results_{wk}.csv'), index_col=0)
    internal_dict[wk] = im[pruned_im.columns]
    break
internal_dict[0].head(4)

,rocksdb.block.cache.miss COUNT,rocksdb.block.cache.hit COUNT,rocksdb.block.cache.add COUNT,rocksdb.block.cache.index.miss COUNT,rocksdb.block.cache.index.hit COUNT,rocksdb.block.cache.index.add COUNT,rocksdb.block.cache.index.bytes.insert COUNT,rocksdb.block.cache.data.miss COUNT,rocksdb.block.cache.data.hit COUNT,rocksdb.block.cache.data.add COUNT,...,rocksdb.bytes.decompressed_P99,rocksdb.bytes.decompressed_P100,rocksdb.bytes.decompressed_COUNT,rocksdb.bytes.decompressed_SUM,rocksdb.db.flush.micros_P50,rocksdb.db.flush.micros_P95,rocksdb.db.flush.micros_P99,rocksdb.db.flush.micros_P100,rocksdb.db.flush.micros_COUNT,rocksdb.db.flush.micros_SUM
index,,,,,,,,,,,,,,,,,,,,,
/home/jieun/data_generation_rocksdb/rocksdb_conf/config1.cnf,1296845,1172926,1213729,0,0,0,0,1296845,1172926,1213729,...,8383.000000,8383.0,1297251,10856979511,21652.173913,30615.000000,30615.0,30615.0,44,1015663
/home/jieun/data_generation_rocksdb/rocksdb_conf/config2.cnf,1304820,4192080,1256074,1582,2701880,1582,5078096,1303238,1490200,1254492,...,14659.000000,14659.0,1304820,19074632333,18135.593220,21979.661017,26561.0,26561.0,63,1074316
/home/jieun/data_generation_rocksdb/rocksdb_conf/config3.cnf,1407632,2195141,1315590,0,0,0,0,1407632,2195141,1315590,...,6578.256132,12483.0,1407897,8847109704,29660.714286,45820.000000,45820.0,45820.0,78,2692570
/home/jieun/data_generation_rocksdb/rocksdb_conf/config4.cnf,1497592,7007880,1252738,974,4003817,974,6993856,1496618,3004063,1251764,...,4385.648850,9451.0,1497592,4718848784,24646.616541,32387.969925,34773.0,34773.0,208,5221234


In [6]:
external_dict = {}
for wk in range(wk_len):
    ex = pd.read_csv(os.path.join(PATH_external, f'external_results_{wk}.csv'), index_col=0)
    external_dict[wk] = ex
    break
external_dict[0].head(4)

,TIME,RATE,WAF,SA
index,,,,
/home/jieun/data_generation_rocksdb/rocksdb_conf/config1.cnf,21.8,4.77,8.7,72.17
/home/jieun/data_generation_rocksdb/rocksdb_conf/config2.cnf,25.9,4.02,8.9,55.83
/home/jieun/data_generation_rocksdb/rocksdb_conf/config3.cnf,52.0,2.00,7.5,34.21
/home/jieun/data_generation_rocksdb/rocksdb_conf/config4.cnf,53.6,1.94,10.5,56.94


## Test Train
- train: 16000, test: 4000

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [8]:
class RocksDBDataset(Dataset):
    def __init__(self, X, y):
        super(RocksDBDataset, self).__init__()
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return (self.X[idx], self.y[idx])

In [9]:
class SingleNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SingleNet, self).__init__()
        self.input_dim = input_dim # 3327
        self.hidden_dim = hidden_dim # 1024
        self.output_dim = output_dim # 148
        self.knob_fc = nn.Sequential(nn.Linear(self.input_dim, self.hidden_dim), nn.ReLU())
#         self.hidden = nn.Sequential(nn.Linear(self.hidden_dim, 64), nn.ReLU())
        self.im_fc = nn.Sequential(nn.Linear(self.hidden_dim, self.output_dim))

    def forward(self, x):
        self.x_kb = self.knob_fc(x)
#         self.h = self.hidden(self.x_kb)
        self.x_im = self.im_fc(self.x_kb)
        return self.x_im

In [10]:
def train(model, train_loader, lr):
    ## Construct optimizer
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    
    ## Set phase
    model.train()
    
    ## Train start
    total_loss = 0.
    for data, target in train_loader:
        ## data.shape = (batch_size, 22)
        ## target.shape = (batch_size, 1)
        ## initilize gradient
        optimizer.zero_grad()
        ## predict
        output = model(data) # output.shape = (batch_size, 1)
        ## loss
        loss = F.mse_loss(output, target)
        ## backpropagation
        loss.backward()
        optimizer.step()
        ## Logging
        total_loss += loss.item()
    total_loss /= len(train_loader)
    return total_loss

def valid(model, valid_loader):
    ## Set phase
    model.eval()
    
    ## Valid start    
    total_loss = 0.
    with torch.no_grad():
        for data, target in valid_loader:
            output = model(data)
            loss = F.mse_loss(output, target) # mean squared error
            total_loss += loss.item()
    total_loss /= len(valid_loader)
    return total_loss

In [11]:
knobOneHot = np.load('knobsOneHot.npy')
internal_m = internal_dict[0]
external_m = external_dict[0]
knobOneHot.shape, internal_m.shape, external_m.shape

((20000, 3327), (20000, 148), (20000, 4))

In [12]:
X_tr, X_te, Im_tr, Im_te, y_tr, y_te, knob_tr, knob_te = \
            train_test_split(knobOneHot, internal_m, external_m, knobs, test_size=0.2, random_state=24)
X_tr.shape, X_te.shape, Im_tr.shape, Im_te.shape, y_tr.shape, y_te.shape, knob_tr.shape, knob_te.shape

((16000, 3327),
 (4000, 3327),
 (16000, 148),
 (4000, 148),
 (16000, 4),
 (4000, 4),
 (16000, 22),
 (4000, 22))

### Pre-training

In [13]:
scaler_X = MinMaxScaler().fit(X_tr) # range: 0~1
scaler_knob = MinMaxScaler().fit(knob_tr)
scaler_Im = MinMaxScaler().fit(Im_tr)
scaler_y = StandardScaler().fit(y_tr)

# X_norm_tr = torch.Tensor(scaler_X.transform(X_tr)).cuda()
# X_norm_te = torch.Tensor(scaler_X.transform(X_te)).cuda()
X_tr = torch.Tensor(X_tr).cuda()
X_te = torch.Tensor(X_te).cuda()
Im_norm_tr = torch.Tensor(scaler_Im.transform(Im_tr)).cuda()
Im_norm_te = torch.Tensor(scaler_Im.transform(Im_te)).cuda()
y_norm_tr = torch.Tensor(scaler_y.transform(y_tr)).cuda()
y_norm_te = torch.Tensor(scaler_y.transform(y_te)).cuda()

Dataset_tr = RocksDBDataset(X_tr, Im_norm_tr)
Dataset_te = RocksDBDataset(X_te, Im_norm_te)

loader_tr = DataLoader(dataset = Dataset_tr, batch_size = 32, shuffle=True)
loader_te = DataLoader(dataset = Dataset_te, batch_size = 32, shuffle=True)

In [14]:
k2i_model = SingleNet(input_dim=X_tr.shape[1], hidden_dim=1024, output_dim=148).cuda()

In [15]:
lr = 0.001
epochs = 30
losses_tr = []

for epoch in range(epochs):
    loss_tr = train(k2i_model, loader_tr, lr)
    losses_tr.append(loss_tr)

    print(f"[{epoch:02d}/{epochs}] loss_tr: {loss_tr}")
        
# print(f"[{epoch:02d}/{epochs}] loss_tr: {loss_tr:.4f}")

[00/30] loss_tr: 0.005664239369798452
[01/30] loss_tr: 0.0025593719827011227
[02/30] loss_tr: 0.0018059247399214654
[03/30] loss_tr: 0.0012160917206201702
[04/30] loss_tr: 0.000908729589311406
[05/30] loss_tr: 0.0007787680691108107
[06/30] loss_tr: 0.0006743897469132207
[07/30] loss_tr: 0.0006050714031443931
[08/30] loss_tr: 0.0005514327536220663
[09/30] loss_tr: 0.0005088210675748997
[10/30] loss_tr: 0.000470154338341672
[11/30] loss_tr: 0.0004443830486852676
[12/30] loss_tr: 0.0004127573323785327
[13/30] loss_tr: 0.0003956633755005896
[14/30] loss_tr: 0.0003793746326700784
[15/30] loss_tr: 0.0003628990884171799
[16/30] loss_tr: 0.0003460286913323216
[17/30] loss_tr: 0.0003339105868944898
[18/30] loss_tr: 0.0003249061446113046
[19/30] loss_tr: 0.00031608836830127984
[20/30] loss_tr: 0.000306517155317124
[21/30] loss_tr: 0.00029595419694669546
[22/30] loss_tr: 0.0002891709686082322
[23/30] loss_tr: 0.0002834960406180471
[24/30] loss_tr: 0.00027454231231240555
[25/30] loss_tr: 0.0002729

### Train with knob2vec

In [16]:
lookup_table = k2i_model.knob_fc[0].weight.T.cpu().detach().numpy()
lookup_table.shape

(3327, 1024)

In [17]:
def get_knob2vec(data, table):
    k2vec = np.zeros((data.shape[0], 22, table.shape[1]))
    for i in range(data.shape[0]):
        idx = (data[i]==1).nonzero().squeeze().cpu().detach().numpy()
        k2vec[i] = lookup_table[idx]
    return k2vec

In [18]:
K2vec_tr = torch.Tensor(get_knob2vec(X_tr, lookup_table)).cuda()
K2vec_tr = torch.reshape(K2vec_tr, (K2vec_tr.shape[0], -1))
K2vec_te = torch.Tensor(get_knob2vec(X_te, lookup_table)).cuda()
K2vec_te = torch.reshape(K2vec_te, (K2vec_te.shape[0], -1))

Dataset_K2vec_tr = RocksDBDataset(K2vec_tr, y_norm_tr)
Dataset_K2vec_te = RocksDBDataset(K2vec_te, y_norm_te)

loader_K2vec_tr = DataLoader(dataset = Dataset_K2vec_tr, batch_size = 32, shuffle=True)
loader_K2vec_te = DataLoader(dataset = Dataset_K2vec_te, batch_size = 32, shuffle=True)

In [19]:
model = SingleNet(input_dim=K2vec_tr.shape[-1], hidden_dim=64, output_dim=4).cuda()

In [20]:
lr = 0.001
epochs = 30
losses_tr = []
losses_te = []
for epoch in range(epochs):
    loss_tr = train(model, loader_K2vec_tr, lr)
    loss_te = valid(model, loader_K2vec_te)
    
    losses_tr.append(loss_tr)
    losses_te.append(loss_te)
    
    print(f"[{epoch:02d}/{epochs}] loss_tr: {loss_tr}\tloss_te:{loss_te:.4f}")

[00/30] loss_tr: 0.21853556246310474	loss_te:0.1536
[01/30] loss_tr: 0.1454527539536357	loss_te:0.1327
[02/30] loss_tr: 0.10852717112749816	loss_te:0.1182
[03/30] loss_tr: 0.08078208340704442	loss_te:0.1168
[04/30] loss_tr: 0.062003011003136634	loss_te:0.1045
[05/30] loss_tr: 0.04747137776762247	loss_te:0.1091
[06/30] loss_tr: 0.04186899189278483	loss_te:0.1213
[07/30] loss_tr: 0.03915024245157838	loss_te:0.1070
[08/30] loss_tr: 0.03755393115803599	loss_te:0.1084
[09/30] loss_tr: 0.035519163811579345	loss_te:0.1067
[10/30] loss_tr: 0.03395218995958567	loss_te:0.1031
[11/30] loss_tr: 0.031224841229617595	loss_te:0.1019
[12/30] loss_tr: 0.030937849586829543	loss_te:0.1038
[13/30] loss_tr: 0.028672505959868432	loss_te:0.1069
[14/30] loss_tr: 0.027426628854125737	loss_te:0.1014
[15/30] loss_tr: 0.028306780805811286	loss_te:0.1002
[16/30] loss_tr: 0.026622327223420145	loss_te:0.1042
[17/30] loss_tr: 0.02584932104870677	loss_te:0.1097
[18/30] loss_tr: 0.024685875909402966	loss_te:0.1081
[19/

### Train with raw knob

In [21]:
knob_norm_tr = torch.Tensor(scaler_knob.transform(knob_tr)).cuda()
knob_norm_te = torch.Tensor(scaler_knob.transform(knob_te)).cuda()

Dataset_knob_tr = RocksDBDataset(knob_norm_tr, y_norm_tr)
Dataset_knob_te = RocksDBDataset(knob_norm_te, y_norm_te)

loader_knob_tr = DataLoader(dataset = Dataset_knob_tr, batch_size = 32, shuffle=True)
loader_knob_te = DataLoader(dataset = Dataset_knob_te, batch_size = 32, shuffle=True)

In [22]:
knob_model = SingleNet(input_dim=knob_norm_tr.shape[1], hidden_dim=16, output_dim=4).cuda()

In [31]:
lr = 0.001
epochs = 30
losses_tr = []
losses_te = []
for epoch in range(epochs):
    loss_tr = train(knob_model, loader_knob_tr, lr)
    loss_te = valid(knob_model, loader_knob_te)
    
    losses_tr.append(loss_tr)
    losses_te.append(loss_te)
    
    print(f"[{epoch:02d}/{epochs}] loss_tr: {loss_tr}\tloss_te:{loss_te:.4f}")

[00/30] loss_tr: 0.19751629308611154	loss_te:0.1772
[01/30] loss_tr: 0.19639316683262587	loss_te:0.1769
[02/30] loss_tr: 0.19647715479135514	loss_te:0.1769
[03/30] loss_tr: 0.19605002128332855	loss_te:0.1762
[04/30] loss_tr: 0.19477278228849174	loss_te:0.1734
[05/30] loss_tr: 0.19171006526052953	loss_te:0.1711
[06/30] loss_tr: 0.18911601109057666	loss_te:0.1686
[07/30] loss_tr: 0.18701285818964242	loss_te:0.1697
[08/30] loss_tr: 0.18560225196182728	loss_te:0.1671
[09/30] loss_tr: 0.18442932121455669	loss_te:0.1661
[10/30] loss_tr: 0.18355640065670015	loss_te:0.1654
[11/30] loss_tr: 0.1831856312677264	loss_te:0.1650
[12/30] loss_tr: 0.18233186051249503	loss_te:0.1651
[13/30] loss_tr: 0.1817149330675602	loss_te:0.1642
[14/30] loss_tr: 0.18187620601803065	loss_te:0.1630
[15/30] loss_tr: 0.1813541674092412	loss_te:0.1635
[16/30] loss_tr: 0.18092557295411826	loss_te:0.1634
[17/30] loss_tr: 0.18055226292461157	loss_te:0.1644
[18/30] loss_tr: 0.18088669869303703	loss_te:0.1635
[19/30] loss_tr

In [33]:
pd.DataFrame(scaler_y.inverse_transform(y_norm_te.cpu().detach().numpy()))

,0,1,2,3
0,31.100000,3.34,9.0,82.090004
1,21.900000,4.74,10.9,98.190002
2,22.500000,4.62,8.3,97.769997
3,34.799999,2.98,7.9,76.199997
4,72.699997,1.43,9.1,48.320000
...,...,...,...,...
3995,26.600000,3.91,8.8,53.279999
3996,66.799995,1.56,12.0,37.780003
3997,91.000000,1.14,10.4,54.340000
3998,83.599998,1.24,8.7,71.419998


In [34]:
pd.DataFrame(scaler_y.inverse_transform(model(K2vec_te).cpu().detach().numpy()))

,0,1,2,3
0,31.542614,3.399611,9.101082,83.414650
1,24.777712,4.598466,9.887324,98.619102
2,29.790953,3.521210,8.572406,96.479355
3,33.720367,3.024567,7.635543,75.495644
4,73.331825,1.308914,8.165498,48.131374
...,...,...,...,...
3995,28.860018,3.774169,8.789579,53.872948
3996,70.283485,1.514679,12.157753,36.780434
3997,83.830109,1.111493,11.106554,53.521385
3998,74.917709,1.161055,8.233262,73.727509


In [28]:
_30 = scaler_y.inverse_transform(knob_model(knob_norm_te).cpu().detach().numpy()) # 30
_30

array([[ 28.582375 ,   3.8760848,   9.241712 ,  81.805565 ],
       [ 28.188007 ,   4.0927906,  11.420867 , 101.219376 ],
       [ 33.180607 ,   3.667392 ,   8.685279 ,  91.01232  ],
       ...,
       [ 81.79058  ,   1.2519226,  10.361152 ,  56.349815 ],
       [ 86.393875 ,   1.3640351,   8.378688 ,  74.18085  ],
       [ 32.761833 ,   3.6919093,   6.8115015,  19.783398 ]],
      dtype=float32)

In [35]:
pd.DataFrame(_30)

,0,1,2,3
0,28.582375,3.876085,9.241712,81.805565
1,28.188007,4.092791,11.420867,101.219376
2,33.180607,3.667392,8.685279,91.012321
3,53.182255,2.951926,7.184204,73.641655
4,82.023170,1.578143,8.407405,50.020275
...,...,...,...,...
3995,27.527189,3.933489,8.395553,54.884876
3996,69.458733,1.892276,12.460921,38.270603
3997,81.790581,1.251923,10.361152,56.349815
3998,86.393875,1.364035,8.378688,74.180847


In [36]:
pd.DataFrame(scaler_y.inverse_transform(knob_model(knob_norm_te).cpu().detach().numpy())) # 60

,0,1,2,3
0,28.627989,3.819625,9.080237,80.633636
1,25.577194,3.950415,11.141011,99.369202
2,31.435863,3.654297,8.461027,89.350555
3,43.789303,3.219799,7.520824,73.618271
4,75.143959,1.423598,8.421802,49.404247
...,...,...,...,...
3995,29.771027,3.828997,8.485855,52.944046
3996,65.110741,1.693185,11.537539,37.955120
3997,90.734459,1.301291,9.891213,54.158501
3998,81.987267,1.408094,8.152787,74.256760
